In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Guardar índices e Target para separar depois
len_train = len(train)
target = train['Transported'].astype(int)
passenger_ids = test['PassengerId'].copy()

# Remover target do treino para concatenar
train_x = train.drop('Transported', axis=1)
df = pd.concat([train_x, test], axis=0).reset_index(drop=True)

print(f"Dataset Total: {df.shape}")

# Extrair Grupo do PassengerId 
df['Group'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
df['GroupSize'] = df['Group'].map(df['Group'].value_counts())

# Extrair Cabine (Deck/Num/Side)
df['Cabin'] = df['Cabin'].fillna('Z/9999/Z')
df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)
df['Num'] = df['Num'].astype(int)

# Voltar nulos originais para 'Z' (ou tratar como Unknown)
df.loc[df['Deck']=='Z', 'Deck'] = np.nan
df.loc[df['Side']=='Z', 'Side'] = np.nan

# Engenharia de Gastos (Consistência CryoSleep)
cols_gastos = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']

# Preencher nulos de gastos com 0
df[cols_gastos] = df[cols_gastos].fillna(0)

# Criar TotalSpend
df['TotalSpend'] = df[cols_gastos].sum(axis=1)
df['NoSpending'] = (df['TotalSpend'] == 0).astype(int)

df.loc[df['TotalSpend'] > 0, 'CryoSleep'] = False

# Preencher HomePlanet baseado no Grupo (Pessoas do mesmo grupo vêm do mesmo planeta)
group_homeplanet = df.groupby('Group')['HomePlanet'].first() # Pega o primeiro não nulo do grupo
df['HomePlanet'] = df['HomePlanet'].fillna(df['Group'].map(group_homeplanet))

# O que sobrar de nulo em Categóricas será preenchido como 'Unknown' ou com a Moda
cat_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        # Preenche com a moda
        df[col] = df[col].fillna(df[col].mode()[0])

# Preencher Idade com Mediana
df['Age'] = df['Age'].fillna(df['Age'].median())

# Extrair Sobrenome
df['Surname'] = df['Name'].fillna('Unknown Unknown').apply(lambda x: x.split(' ')[-1])
df['FamilySize'] = df['Surname'].map(df['Surname'].value_counts())
# Se sobrenome for Unknown, FamilySize não é confiável (setar para 0 ou média)
df.loc[df['Surname']=='Unknown', 'FamilySize'] = 0

# Deck + Side (Mapeia local específico do navio)
df['DeckSide'] = df['Deck'].astype(str) + df['Side'].astype(str)

# Log Transform dos gastos
for col in cols_gastos + ['TotalSpend']:
    df[col] = np.log1p(df[col])

# Colunas que vamos usar no modelo
# Removemos IDs e textos livres que já foram processados
cols_drop = ['PassengerId', 'Name', 'Cabin', 'Group', 'Surname']
df_final = df.drop(columns=cols_drop)

# Separar de volta em Treino e Teste
X_train = df_final.iloc[:len_train].copy()
X_test = df_final.iloc[len_train:].copy()
y_train = target

# Identificar Categóricas para o CatBoost
cat_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side', 'DeckSide']

# Garantir que são strings
for col in cat_features:
    X_train[col] = X_train[col].astype(str)
    X_test[col] = X_test[col].astype(str)

print("Dados Processados!")
print(f"Treino: {X_train.shape} | Teste: {X_test.shape}")


print("Iniciando Treinamento CatBoost...")

model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.03,
    depth=6,
    eval_metric='Accuracy',
    verbose=200,
    random_seed=42
)

# Treinar no dataset completo (Full Train) para maximizar aprendizado
model.fit(X_train, y_train, cat_features=cat_features)


print("Gerando previsões...")
predictions = model.predict(X_test)
# CatBoost retorna 0 e 1, converter para Bool (True/False)
predictions_bool = predictions.astype(bool)

submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Transported': predictions_bool
})

submission.to_csv('submission_advanced_eda_catboost.csv', index=False)
print("Arquivo 'submission_advanced_eda_catboost.csv' gerado com sucesso!")

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split


train = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\data\train.csv')
test = pd.read_csv(r'C:\Users\João Pedro\Documents\UFG\AMS\AS2\data\test.csv')

len_train = len(train)
target = train['Transported'].astype(int)
passenger_ids = test['PassengerId'].copy()

train_x = train.drop('Transported', axis=1)
df = pd.concat([train_x, test], axis=0).reset_index(drop=True)

print(f"Dataset Total: {df.shape}")

# Extração Básica (Cabin e Grupos) 
df['Group'] = df['PassengerId'].apply(lambda x: x.split('_')[0])
df['GroupSize'] = df['Group'].map(df['Group'].value_counts())

# Usamos -1 para nulos numéricos (árvores lidam bem com isso) e 'Unknown' para texto
df['Cabin'] = df['Cabin'].fillna('Unknown/-1/Unknown')
df[['Deck', 'Num', 'Side']] = df['Cabin'].str.split('/', expand=True)
df['Num'] = pd.to_numeric(df['Num'], errors='coerce').fillna(-1).astype(int)

# Engenharia de Gastos 
cols_gastos = ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
df[cols_gastos] = df[cols_gastos].fillna(0)

# Features de Estilo de Gastos
df['LuxurySpend'] = df['RoomService'] + df['Spa'] + df['VRDeck']
df['BasicSpend'] = df['FoodCourt'] + df['ShoppingMall']
df['TotalSpend'] = df[cols_gastos].sum(axis=1)
df['NoSpending'] = (df['TotalSpend'] == 0).astype(int)

# Consistência
df.loc[df['TotalSpend'] > 0, 'CryoSleep'] = False
df.loc[df['CryoSleep'] == True, cols_gastos] = 0

# Imputação Inteligente
group_homeplanet = df.groupby('Group')['HomePlanet'].first()
df['HomePlanet'] = df['HomePlanet'].fillna(df['Group'].map(group_homeplanet))

cat_cols = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side']
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].mode()[0])

df['Age'] = df['Age'].fillna(df['Age'].median())

# Features Familiares
df['Surname'] = df['Name'].fillna('Unknown Unknown').apply(lambda x: x.split(' ')[-1])
df['FamilySize'] = df['Surname'].map(df['Surname'].value_counts())
df.loc[df['Surname']=='Unknown', 'FamilySize'] = 0

df['DeckSide'] = df['Deck'].astype(str) + df['Side'].astype(str)

# Cria 10 regiões baseadas na numeração
df['CabinRegion'] = pd.qcut(df['Num'].replace(-1, np.nan), q=10, labels=False).fillna(-1).astype(int)

# Log Transform dos gastos
cols_log = cols_gastos + ['TotalSpend', 'LuxurySpend', 'BasicSpend']
for col in cols_log:
    df[col] = np.log1p(df[col])

cols_drop = ['PassengerId', 'Name', 'Cabin', 'Group', 'Surname']
df_final = df.drop(columns=cols_drop)

X = df_final.iloc[:len_train].copy()
X_test_final = df_final.iloc[len_train:].copy()
y = target

# Identificar Categóricas
cat_features = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Deck', 'Side', 'DeckSide']
for col in cat_features:
    X[col] = X[col].astype(str)
    X_test_final[col] = X_test_final[col].astype(str)

print("Separando Treino e Validação para Early Stopping...")

# Aqui separamos 15% do treino original para servir de "fiscal" do modelo
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15, random_state=42)

print("Iniciando Treinamento CatBoost Otimizado...")

model = CatBoostClassifier(
    iterations=2000,
    learning_rate=0.02,     
    depth=6,                # Profundidade balanceada
    l2_leaf_reg=3.0,        # Regularização padrão
    loss_function='Logloss',
    eval_metric='Accuracy',
    verbose=200,            # Imprime a cada 200 iterações
    random_seed=42,
    od_type='Iter',         # Detecção de Overfitting
    od_wait=100             # Espera 100 iterações sem melhorar antes de parar
)

# Treinamos observando o conjunto de validação
model.fit(
    X_train, y_train, 
    cat_features=cat_features,
    eval_set=(X_val, y_val), # O modelo usa isso para saber quando parar
    use_best_model=True      # Garante que no final ele volta para a melhor época
)

print(f"Melhor Acurácia na Validação: {model.get_best_score()['validation']['Accuracy']:.4f}")

# Previsão
print("Gerando previsões no Teste...")
predictions = model.predict(X_test_final)
predictions_bool = predictions.astype(bool)

submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Transported': predictions_bool
})

submission.to_csv('submission_catboost_improved.csv', index=False)
print("Arquivo 'submission_catboost_improved.csv' gerado com sucesso!")

Dataset Total: (12970, 13)
Separando Treino e Validação para Early Stopping...
Iniciando Treinamento CatBoost Otimizado...


C:\Users\João Pedro\AppData\Local\Temp\ipykernel_1664\2539684284.py:56: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[col].mode()[0])


0:	learn: 0.7841386	test: 0.7745399	best: 0.7745399 (0)	total: 236ms	remaining: 7m 50s
200:	learn: 0.8170253	test: 0.7898773	best: 0.7898773 (200)	total: 13.1s	remaining: 1m 56s
400:	learn: 0.8297469	test: 0.7983129	best: 0.7990798 (393)	total: 27.6s	remaining: 1m 50s
600:	learn: 0.8436866	test: 0.8044479	best: 0.8052147 (591)	total: 39.4s	remaining: 1m 31s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.8067484663
bestIteration = 674

Shrink model to first 675 iterations.
Melhor Acurácia na Validação: 0.8067
Gerando previsões no Teste...
Arquivo 'submission_catboost_improved.csv' gerado com sucesso!


In [ ]:
import optuna
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

print("Preparando dados para Optuna...")

def objective(trial):
    # Definindo o espaço de busca (Hiperparâmetros)
    params = {
        'iterations': 2000, # Mantemos fixo alto, usamos early_stopping
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'depth': trial.suggest_int('depth', 4, 8), # Árvores profundas (>8) costumam overfittar aqui
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10), # Regularização
        'random_strength': trial.suggest_float('random_strength', 0.1, 10), # Ajuda a ignorar ruído
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'border_count': 254,
        'loss_function': 'Logloss',
        'eval_metric': 'Accuracy',
        'verbose': False,
        'random_seed': 42,
        'od_type': 'Iter',
        'od_wait': 50, # Early Stopping
        'cat_features': cat_features # Sua lista de colunas categóricas
    }

    # Cross-Validation com 5 Folds
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    for train_index, val_index in skf.split(X_train, y_train):
        X_tr, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

        model = CatBoostClassifier(**params)
        
        # Treina e avalia
        model.fit(X_tr, y_tr, 
                  eval_set=[(X_val, y_val)], 
                  early_stopping_rounds=50, 
                  verbose=False)
        
        # Pega a melhor acurácia desse fold
        # best_score_ retorna um dicionário, pegamos a validação
        best_acc = model.get_best_score()['validation']['Accuracy']
        scores.append(best_acc)

    # O objetivo é maximizar a MÉDIA das acurácias dos 5 folds
    return np.mean(scores)

print("Iniciando busca com Optuna (Isso pode demorar alguns minutos)...")
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30) 

print("\n" + "="*50)
print("MELHORES PARÂMETROS ENCONTRADOS:")
print("="*50)
print(study.best_params)
print(f"Melhor Acurácia Média (CV): {study.best_value:.5f}")

# Treinar o modelo final com os melhores parâmetros
print("\nTreinando modelo final com os parâmetros vencedores...")

# Pegar os melhores parâmetros
final_params = study.best_params
final_params['iterations'] = 2500 # Aumentamos um pouco para o treino final total
final_params['cat_features'] = cat_features
final_params['verbose'] = 100
final_params['od_wait'] = 100

# Treinar no Dataset completo
model_final = CatBoostClassifier(**final_params)
model_final.fit(X_train, y_train) 



In [ ]:
# Prever
print("Gerando previsões...")
predictions = model_final.predict(X_test_final) 
predictions_bool = predictions.astype(bool)

# Salvar
submission = pd.DataFrame({
    'PassengerId': passenger_ids, 
    'Transported': predictions_bool
})

submission.to_csv('submission_catboost_optuna_v2.csv', index=False)
print("Arquivo 'submission_catboost_optuna_v2.csv' gerado! Boa sorte!")

Gerando previsões...
Arquivo 'submission_catboost_optuna_v2.csv' gerado! Boa sorte!
